In [1]:
%load_ext autoreload

In [8]:
%autoreload 2
import os
import sys
from pathlib import Path
import pyperclip
from src.story import create_html_challenges

chat_app_dir = Path().absolute().parent / "chat-app"
assert chat_app_dir.exists()
if (str(chat_app_dir)) not in sys.path:
    sys.path.append(str(chat_app_dir))
from src.config_loader import config
from src.utils import anthropic_generate, upload_to_gcs, load_json


In [3]:
STORY_NAME = "story_camping_trip_gone_awry"
notebook_dir = Path().absolute()  # This gives src/notebooks
phrase_dir = notebook_dir.parent / "data" / "phrases" #where we store text files of phrases
story_dir = notebook_dir.parent / "outputs" / "stories" / STORY_NAME 
chat_dir = notebook_dir.parent / "chat"
story_dialgoue_file = story_dir / f"{STORY_NAME}.json"

In [4]:
story_dialogue = load_json(story_dialgoue_file)

In [16]:
def get_challenge_generation_prompt(story_dialogue: dict) -> str:
    """
    Generate a prompt for an LLM to create language learning scenarios where learners
    must discover and handle complications.
    
    Args:
        story_dialogue: Dictionary containing dialogue parts ("setup", "resolution" etc)
        
    Returns:
        str: Formatted prompt for the LLM
    """
    
    dialogue_text = []
    for part in story_dialogue.values():
        if "dialogue" in part:
            for utterance in part["dialogue"]:
                dialogue_text.append(f'{utterance["speaker"]}: {utterance["text"]}')
    
    story_context = "\n".join(dialogue_text)
    
    prompt = f"""Analyze this dialogue and create 5 practical language learning scenarios.
Each scenario should have a main challenge where the learner must discover why their request cannot be fulfilled immediately.
Base the scenarios on this story context:

{story_context}

Create the scenarios in this JSON format:
{{
    "scenarios": [
        {{
            "role": "who the teacher will roleplay (e.g. restaurant staff)",
            "context": "brief setting description",
            "challenge": "overall task to complete (e.g. 'Book a table for {{time}}')", 
            "question": "What problem or complication prevents your request? Why isn't it possible?",
            "variables": {{
                "time": ["6:30", "7:00", "7:30"]
            }},
            "complications": [
                "I'm sorry, we're fully booked at {{time}}, but we have space an hour later",
                "Unfortunately our kitchen closes early today, before {{time}}",
                "We only have outdoor tables available at {{time}}, and it's expected to rain"
            ],
            "success_criteria": "specific condition showing challenge completion (e.g. 'Learner understands the complication and either accepts an alternative time or finds another solution')"
        }}
    ]
}}

Requirements:
1. Create 5 common scenarios (restaurant, hotel, shop, etc.)
2. Each scenario should have:
   - A practical main challenge
   - The standard question about discovering the complication
   - One simple variable (time, item, day, etc.)
   - Three realistic complications that use the variable
3. Complications should:
   - Clearly explain why the original request can't be fulfilled
   - Suggest or hint at possible alternatives
   - Use the variable naturally
4. Success criteria should:
   - Focus on understanding the problem
   - Allow for multiple solutions
   - Include accepting alternatives

Remember:
- The question is always about discovering why the request can't be fulfilled
- Complications are the answers that will be revealed
- Keep language practical and everyday
- Every complication should suggest a way forward
- Success means understanding the problem and finding a solution

Output only the JSON with no additional text."""

    return prompt

In [17]:
prompt = get_challenge_generation_prompt(story_dialogue)

In [18]:
pyperclip.copy(prompt)

In [17]:

# Example challenges for testing
test_challenges = [
    {
        "challenge_description": "You are at a coffee shop, find out the server's name.",
        "llm_prompt": """You are a barista at a coffee shop. Your name is Willow.""",
        "answer": "The person's name is Willow"
    },
    {
        "challenge_description": "Order a coffee in Swedish and ask how much it costs",
        "llm_prompt": """You are a role playing Swedish café worker for language learning. The learners has a challenge to order a coffee and find the price (they cost 10 kronor).
        Respond naturally in Swedish; guiding the learner (using swedish) if the make a mistake.""",
        "answer": "The coffee costs 10 kronor (price in Swedish: 'Det kostar tio kronor')"
    },
    {
    "challenge_description": "Order 2 beers in Russian and find out what the waitress thinks of your friend",
    "llm_prompt": """IMPORTANT: Respond ONLY in Russian, no English.
    
    You are a waitress at a Russian bar. When customers order drinks, respond only in Russian. If they ask about their friend, say "очень красивый" (very handsome). If they make mistakes, guide them in Russian.
    
    DO NOT use any English - maintain Russian immersion.""",
    "answer": "The waitress thinks the friend is very handsome (in Russian: 'очень красивый')"
},
    {
    "challenge_description": "Order a massive cake in Spanish",
    "llm_prompt": """IMPORTANT: Respond ONLY in Spanish, no English.
    
    You are a waitress at a Spanish cafe. Respond to the customer request, they are trying to order cake. Guide them to the correct use of Spanish if they make a mistake.
    
    DO NOT use any English - maintain Spanish immersion.""",
    "answer": "None really"
}
]


# Create the HTML file with test challenges
create_html_challenges(
    challenges=test_challenges,
    output_dir=chat_dir,
    title="DuoLaingo Conversation Practice",
    language="English and Swedish"
)

HTML challenges created at: y:\Python Scripts\audio-language-trainer\chat\duolaingo_conversation_practice.html


In [18]:
upload_to_gcs(file_path=chat_dir / "duolaingo_conversation_practice.html", bucket_prefix="test_challenges", )

'https://storage.googleapis.com/audio-language-trainer-stories/test_challenges/duolaingo_conversation_practice.html'